Cell 1 loads up all the libraries we are using. In the first lessons, we used fastAI, while in this one we use Tensorflow with Keras. Both of these are large libraries, and its designers decided to allow programmers to load only the pieces that are relevant, which is what the 'from keras.XXX import YYY' lines do--they load just the partrt of Keras (or other library) that we want. This makes the load time faster and the final running program smaller.

The second half of this cell does the familiar step of loading the data sets that I have prepared, which in this case are a series of abdominal CTs as well as hand-traced masks of the Pancreas, courtesy of The Cancer Image Archive: http://TheCancerImageArchive....

In [1]:
#Cell 1

!pip install -q keras
!pip install natsort
import numpy as np
import numpy.ma as ma
import os
import shutil
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Dense, Dropout, Activation, Flatten, BatchNormalization, Reshape
from keras.engine.topology import Layer

from keras.layers.merge import concatenate, add
from keras.layers.core import Lambda
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import imageio
from natsort import natsorted
import random
import warnings
warnings.filterwarnings("ignore")



!rm -rf ./MC4-TensorflowUNet

!rm -rf trainimages
!mkdir trainimages
!rm -rf trainmasks
!mkdir trainmasks

!rm -rf validationimages
!mkdir validationimages
!rm -rf validationmasks
!mkdir validationmasks

!rm -rf testimages
!mkdir testimages
!rm -rf testmasks
!mkdir testmasks


!rm *.zip
#Zip files 1 & 2 have the data used for training
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pH5FgRRUPCmzbCszm101vAyTVe7Ufs60' -O ./Pt1.zip
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V4zAE19E1kLUK0Z0YIWNpMq0K8lOARtu' -O ./Pt2.zip

!unzip -q -o "./Pt1.zip"
!unzip -q -o "./Pt2.zip"
!mv *-Mask.jpg ./trainmasks
!mv *.jpg ./trainimages

# Zip3 is for validation
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=18mVgpUeghNOHKMazHLINyjq6lPKstLJp' -O ./Pt3.zip
!unzip -q -o "./Pt3.zip"
!mv *-Mask.jpg ./validationmasks
!mv *.jpg ./validationimages

#Part 4 is for testing
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Rh53wMdhEvOOel6z2c47uPW_R72I8dLe' -O ./Pt4.zip
!unzip -q -o "./Pt4.zip"
!mv *-Mask.jpg ./testmasks
!mv *.jpg ./testimages

Using TensorFlow backend.


rm: cannot remove '*.zip': No such file or directory
--2019-11-21 17:08:15--  https://docs.google.com/uc?export=download&id=1pH5FgRRUPCmzbCszm101vAyTVe7Ufs60
Resolving docs.google.com (docs.google.com)... 74.125.31.101, 74.125.31.100, 74.125.31.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.31.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-60-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gdgc11m90tdpr65uiiq0ael0hn86pptp/1574352000000/16160187475894979440/*/1pH5FgRRUPCmzbCszm101vAyTVe7Ufs60?e=download [following]
--2019-11-21 17:08:22--  https://doc-0s-60-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gdgc11m90tdpr65uiiq0ael0hn86pptp/1574352000000/16160187475894979440/*/1pH5FgRRUPCmzbCszm101vAyTVe7Ufs60?e=download
Resolving doc-0s-60-docs.googleusercontent.com (doc-0s-60-docs.googleusercontent.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Connecting to

#Data Wrangling
Here we organize the data into training, validation and testing sets, with each having an image and mask, so 6 directories total. I started with the TCIA-Pancreas data set, which has hand-traced contours of the pancreas.
I took each image that had some pancreas traced, applied 400/40 W/L settings, converted to 8 bits, and then cropped to the central 256x256 (just to reduce memory demands). The mask is also cropped to match, and is either o or 1.
THese are then loaded into the respective memory arrays.

In [2]:
# Cell 2

train_X = []
train_Y = []
val_X = []
val_Y = []
test_X = []
test_Y = []

im_list = natsorted(os.listdir('./trainimages'))
for f in im_list:
  pth = './trainimages/' + f 
  img = imageio.imread(pth)
  train_X.append(np.array(img))
train_X = np.array(train_X)
train_X = train_X/255
train_X = train_X[..., np.newaxis]

im_list = natsorted(os.listdir('./trainmasks'))
for f in im_list:
  pth = './trainmasks/' + f 
  img = imageio.imread(pth)
  train_Y.append(np.array(img))
train_Y = np.array(train_Y)
train_Y = (train_Y > 0).astype(np.float32)
train_Y = train_Y[..., np.newaxis]

im_list = natsorted(os.listdir('./validationimages'))
for f in im_list:
  pth = './validationimages/' + f 
  img = imageio.imread(pth)
  val_X.append(np.array(img))
val_X = np.array(val_X)
val_X = val_X/255
val_X = val_X[..., np.newaxis]

im_list = natsorted(os.listdir('./validationmasks'))
for f in im_list:
  pth = './validationmasks/' + f 
  img = imageio.imread(pth)
  val_Y.append(np.array(img))
val_Y = np.array(val_Y)
val_Y = (val_Y > 0).astype(np.float32)
val_Y = val_Y[..., np.newaxis]

im_list = natsorted(os.listdir('./testimages'))
for f in im_list:
  pth = './testimages/' + f 
  img = imageio.imread(pth)
  test_X.append(np.array(img))
test_X = np.array(test_X)
test_X = test_X/255
test_X = test_X[..., np.newaxis]

im_list = natsorted(os.listdir('./testmasks'))
for f in im_list:
  pth = './testmasks/' + f 
  img = imageio.imread(pth)
  test_Y.append(np.array(img))
test_Y = np.array(test_Y)
test_Y = (test_Y > 0).astype(np.float32)
test_Y = test_Y[..., np.newaxis]

print(train_X.shape[0],"images for training,", val_X.shape[0], "images for validation, and", test_X.shape[0], "images for testing")

WIDTH = train_X.shape[2]
HEIGHT = train_X.shape[1]
CHANNELS = 1
#print ('X and Y dims are ' + str(WIDTH) + 'x' + str(HEIGHT))

1976 images for training, 260 images for validation, and 143 images for testing


#Cost Functions
A critical element of every machine learning algorithm is to select a good cost function. In the classificaiton problems of prior articles, we used the accuracy of predictions as the cost function. For segmentation, one potentially could compute a score for teh classification of each pixel (object or not object), but a more common metric is the Dice Similiarity Score (a.k.a. Dice Score). This essentially means the amount of overlap between the gold standard and the prediction. If they perfectly agree, the Dice Score is 1, and if there is no overlap, the score is 0. We can convert the Dice Score toa Dice Loss by subtracting it from 1. One can also convert the Dice Score to a cross-entropy function, and this can work well in some cases. Code for both is provided, and you are encouraged to try both cost functions to see the effect. Note, however, that the above can result in Dice values of zero, and we don't want to divide by zero. In fact, we don't want to divide by a small number, so we add 'smooth' to the value (in our case we are using 1) so the values of Dice actually are 1 to 2, not 0 to 1. And that means the Dice Loss will be -1 to 0, but the loss reported is the sum, so it will have a larger range.


In [0]:
# Cell 3
def dice_coeff(y_true, y_pred):
    _epsilon = 10 ** -7
    intersections = tf.reduce_sum(y_true * y_pred)
    unions = tf.reduce_sum(y_true + y_pred)
    dice_scores = (2.0 * intersections + _epsilon) / (unions + _epsilon)
    return dice_scores

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss
  
get_custom_objects().update({"dice": dice_loss})

class LayerNormalization (Layer) :
    
    def call(self, x, mask=None, training=None) :
        axis = list (range (1, len (x.shape)))
        x /= K.std (x, axis = axis, keepdims = True) + K.epsilon()
        x -= K.mean (x, axis = axis, keepdims = True)
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape

#Building the U-Net
We are finally ready to build our Network. One difference of Keras versus the previous examples of FastAI is that Keras is designed so that there is one line of code for each layer of the network (plus the setup and training/evlauation code). This means that for a moderately complex network like a ResNet34 classifier, there would be 34 lines of code. U-Nets are popular network architectures for segmentation. These get their name because of the unique way they are built: The first part produces repeated reductions in resolution as the 'important' parts of the image are retained that reflect the structures that it is trained to recognize. Typically there are 4 or 5 such reduction layers that each consist of convultions, ReLUs, and Pooling layers. Once the image is reduced to the critical components, the networks begins to reconstruct the precise margins of the critical elements by using 'skip layers'. As the resolution is stored using convolutional-transpose layers, the layers 'look' back to the layers where the resolution was reduced to try to best define the margins of the structures. See Ronneberger (Ronneberger, Olaf; Fischer, Philipp; Brox, Thomas (2015). "U-Net: Convolutional Networks for Biomedical Image Segmentation". arXiv:1505.04597  https://arxiv.org/pdf/1505.04597.pdf).

There is a lot more going on that I will only provide references to (these will be covered in the future):
1. ReLU Activation Function: https://arxiv.org/pdf/1502.01852.pdf
2. Kernel Intialization (the weights loaded into the network before anything is done): https://arxiv.org/pdf/1502.01852.pdf
3. Transpose Convolution: https://medium.com/apache-mxnet/transposed-convolutions-explained-with-ms-excel-52d13030c7e8#Building the U-Net


In [0]:
# Cell 4
# this function defines the U-Net model
def build_model(act_fn = 'relu', init_fn = 'he_normal'): 
# this takes in a 256 x 256 pixel image with 1 channel (gray scale)
    inputs = Input((256,256,1))
# filt is the number of filters in first layer, and each is doubled. 
# conv is the size of the first convolution kernel. All after are 3x3
    filt = 6
    conv = 7
    # note we use linear function before layer normalization
    conv1 = Conv2D(filt, conv, activation = 'linear', padding = 'same', kernel_initializer = init_fn)(inputs)
    conv1 = LayerNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(filt * 2, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(filt * 4, 3, activation = 'linear', padding = 'same', kernel_initializer = init_fn)(pool2)
    conv3 = LayerNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(filt * 8, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(pool3)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(filt * 8, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(pool4)

    up6 = Conv2D(filt * 8, 2, activation = 'linear', padding = 'same', kernel_initializer = init_fn)(UpSampling2D(size = (2,2))(conv5))
    up6 = LayerNormalization()(up6)
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(filt * 8, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(merge6)

    up7 = Conv2D(filt * 4, 2, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(filt * 4, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(merge7)

    up8 = Conv2D(filt * 2, 2, activation = 'linear', padding = 'same', kernel_initializer = init_fn)(UpSampling2D(size = (2,2))(conv7))
    up8 = LayerNormalization()(up8)
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(filt * 2, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(merge8)

    up9 = Conv2D(filt, 2, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(filt, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(merge9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'dice', metrics=[dice_coeff])
    return model



#Train it
Now that our model is defined, we are (finally!) ready to build it and train it. Trial and error with colab hardware (K80) has shown that a batch_size of 64 works, and that we can get reasonable results after just a few epochs (but you can train longer to get better results)

In [5]:
# Cell 5
#create the U-Net
model = build_model()
# checkpointer will save the model, in this case any time there is a better error
checkpointer = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

#  I get 20 seconds per epoch on colab notebook (K80s). so 50 epochs is under 20 minutes
# It will take more time and more examples to get pretty good results, but this gets reasonable enough considering difficulty of pancreas
# reduce the number of epochs if you can't wait 20 minutes. It will just reduce the accuracy of the segmentation
epochs = 50
batch_size = 64
results = model.fit(train_X, train_Y, validation_data=(val_X, val_Y), batch_size=batch_size, epochs=epochs, callbacks=[checkpointer])












Train on 1976 samples, validate on 260 samples
Epoch 1/50





1976/1976 [==============================] - 15s 7ms/step - loss: 0.9552 - dice_coeff: 0.0448 - val_loss: 0.9459 - val_dice_coeff: 0.0541

Epoch 00001: val_loss improved from inf to 0.94590, saving model to model.h5
Epoch 2/50
1976/1976 [==============================] - 5s 3ms/step - loss: 0.9409 - dice_coeff: 0.0591 - val_loss: 0.9231 - val_dice_coeff: 0.0769

Epoch 00002: val_loss improved from 0.94590 to 0.92308, saving model to model.h5
Epoch 3/50
1976/1976 [==============================] - 5s 3ms/step - loss: 0.8828 - dice_coeff: 0.1172 - val_loss: 0.7968 - val_dice_coeff: 0.2032

Epoch 00003: val_loss improved from 0.92308 to 0.79684, saving model to model.h5
Epoch 4/50
1976/1976 [==============================] - 5s 3ms/step - loss: 0.7475 - dice_coeff: 0.2525 - val_loss: 0.6721 - val_dice_coeff: 0.3279

Epoch 00004: val_loss improved from 0.79684 to 0.67211, saving model to model.h5
Epoch 5/50
1976/1976 

#Test it!
Now that the model is trained, we can test our segmentation tool on the test holdout cases. Note that the prediction is the probability that a pixel is object (pancreas) or not object (anything other than pancreas). We use 0.5 as the threshold for deciding Pancreas or not, though again you can adjust this if your task might view it as more valuable to include non-pancreas pixels in order to reduce the number of missed pancreas pixels.
Note also that we calculate the Dice Score. The Dice calculated in training is not the actual Dice score, but it is proportional (we make adjustments so it is never 0 or have a divide by 0 error), and therefore works.

In [6]:
# Cell 6
# load our trained model
model.load_weights("./model.h5")
preds_test = model.predict(test_X, verbose=1)
preds_test = (preds_test > 0.5).astype(np.uint8)

def np_dice(true, pred):
    intersection = np.sum(true * pred)
    dc =(2.0 * intersection) / (np.sum(true) + np.sum(pred))
    return dc


print ("=============================================================================================================")
print("The dice score for test cases is: ", np_dice(test_Y, preds_test))

143/143 [==============================] - 1s 4ms/step
The dice score for test cases is:  0.5306023654533786
